In [35]:
import redis
import time
import pickle

In [ ]:
def getLUT() :
    return {}

In [ ]:
def computeAttack(tests) :
    lut = getLUT()
    return lut[tests]

In [37]:
pool = redis.ConnectionPool(host="localhost", port=6379, db=0)
r = redis.Redis(connection_pool=pool)
p = r.pubsub()

p.subscribe("Queue")

p.get_message()
time.sleep(0.5)
def getData(p) :
    for mes in p.listen() :
        print(pickle.loads(mes["data"]))


In [ ]:
for tests in getData(p) :
    r.publish(  channel="Log", 
                message=pickle.dumps(
                    (   
                        tests[1], 
                        computeAttack(tests[1])
                    )
                )
            )